# Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.

## **Design a pipeline that includes the following steps:**
> ## Use an automated feature selection method to identify the important features in the dataset  
> ## Create a numerical pipeline that includes the following steps
> ## Impute the missing values in the numerical columns using the mean of the column values  
> ## Scale the numerical columns using standardisation  
> ## Create a categorical pipeline that includes the following steps:
> ## Impute the missing values in the categorical columns using the most frequent value of the column  
> ## One-hot encode the categorical columns  
> ## Combine the numerical and categorical pipelines using a ColumnTransformer  
> ## Use a Random Forest Classifier to build the final model  
## **Evaluate the accuracy of the model on the test dataset**

## `Note:` Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements for the pipeline.
___

>## 1. Importting Necessary Modules & Packages

In [964]:
import pandas as pd
import numpy as np
from seaborn import load_dataset
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from warnings import filterwarnings as fw
fw('ignore')

>## 2. Importing `tips` Dataset

In [965]:
tips = load_dataset('tips')

In [966]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [967]:
tips.shape

(244, 7)

In [968]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


### `Missing Values`

In [969]:
tips.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

>## Automated Feature Selection

In [970]:
def feature_selection(data):  
    # Label encoding categorical variables
    data = data.drop(['smoker'],axis=1)
    categorical_features = data.select_dtypes(include='category').columns.tolist()    
    X_categorical = data[categorical_features]
    target = tips['smoker']
    encoder = LabelEncoder()
    encoded_features = X_categorical.apply(encoder.fit_transform)
    
    # Select the best categorical features using mutual_info_classif 
    mi_scores = mutual_info_classif(encoded_features,target,random_state=0)
    feature_scores = pd.DataFrame({'Feature': categorical_features, 'Score': mi_scores})   
    
    # Select the best numerical features using Variance Threshold
    numerical_features = data.select_dtypes(include=['int64','float64'])
    selector_numerical = VarianceThreshold(threshold=0.3).fit(numerical_features)
    
    # Merge the selected features
    selected_categorical = feature_scores.loc[feature_scores['Score'] > 0.02, 'Feature'].tolist()    
    selected_numerical = numerical_features.columns[selector_numerical.get_support()].tolist()    
    return selected_categorical,selected_numerical

In [971]:
categorical,numerical = feature_selection(tips)

In [972]:
categorical,numerical

(['sex', 'day'], ['total_bill', 'tip', 'size'])

In [973]:
best_features = pd.concat([tips[categorical],tips[numerical],tips.smoker],axis=1)

### `Encoding Target Variable Using Label Encoder`

In [974]:
encoder = LabelEncoder()
best_features.smoker = encoder.fit_transform(tips.smoker)

>## 3. Numerical Processor Pipeline

In [975]:
numerical_processor = Pipeline(
    steps = [('Mean_Imputer',SimpleImputer(missing_values=np.nan,strategy='mean')),
             ('Scaling',StandardScaler())])

In [976]:
numerical_processor

Pipeline(steps=[('Mean_Imputer', SimpleImputer()),
                ('Scaling', StandardScaler())])

>## 4. Categorical Processor Pipeline

In [977]:
categorical_processor = Pipeline(
    steps = [('Most_frequent_Imputer',SimpleImputer(missing_values=np.nan,strategy='most_frequent')),
              ('Encoding',OneHotEncoder())])

In [978]:
categorical_processor

Pipeline(steps=[('Most_frequent_Imputer',
                 SimpleImputer(strategy='most_frequent')),
                ('Encoding', OneHotEncoder())])

>## 5. Combining Both Processors Into Single Pipeline Using ColumnTransformer

In [979]:
Processor=ColumnTransformer(
    [('Numerical',numerical_processor,numerical),
     ('categorical',categorical_processor,categorical)
    ])

In [980]:
Processor

ColumnTransformer(transformers=[('Numerical',
                                 Pipeline(steps=[('Mean_Imputer',
                                                  SimpleImputer()),
                                                 ('Scaling',
                                                  StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('categorical',
                                 Pipeline(steps=[('Most_frequent_Imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoding',
                                                  OneHotEncoder())]),
                                 ['sex', 'day'])])

>## 6. RandomForestClassifier Pipeline

In [981]:
rfc_pipe = make_pipeline(Processor,RandomForestClassifier(random_state=1))

In [982]:
rfc_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('Numerical',
                                                  Pipeline(steps=[('Mean_Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  ['total_bill', 'tip',
                                                   'size']),
                                                 ('categorical',
                                                  Pipeline(steps=[('Most_frequent_Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoding',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'day'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1))])

>## 7. Train-Test Split

In [983]:
X = tips.drop('smoker',axis=1)
y = tips.smoker

In [984]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

>## 8. Model Training

In [985]:
rfc_pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('Numerical',
                                                  Pipeline(steps=[('Mean_Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  ['total_bill', 'tip',
                                                   'size']),
                                                 ('categorical',
                                                  Pipeline(steps=[('Most_frequent_Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoding',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'day'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1))])

>## 9. Model Performance

In [986]:
y_pred = rfc_pipe.predict(X_test)

In [987]:
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.7551020408163265
[[29  6]
 [ 6  8]]
              precision    recall  f1-score   support

          No       0.83      0.83      0.83        35
         Yes       0.57      0.57      0.57        14

    accuracy                           0.76        49
   macro avg       0.70      0.70      0.70        49
weighted avg       0.76      0.76      0.76        49



>## 10. Result Interpretation
### The `rfc_pipe` accuracy is 75.5 % which can be improved by using hyperparameter tuning.

# Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.
___

# `Importing necessary Librariies & Modules`

In [988]:
from seaborn import load_dataset
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from warnings import filterwarnings as fw
fw('ignore')

# `Importing iris Dataset`

In [989]:
iris = load_dataset('iris')

In [990]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [991]:
iris.shape

(150, 5)

In [992]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## `Missing Values`

In [993]:
iris.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

## `Encoding Target Variable Using Label Encoder`

In [994]:
encoder = LabelEncoder()
iris.species = encoder.fit_transform(iris.species)

# `Numerical Processor Pipeline`

In [995]:
Processor = Pipeline(
    steps = [('Mean_Imputer',SimpleImputer(missing_values=np.nan,strategy='mean')),
              ('Scaling',StandardScaler())])

In [996]:
Processor

Pipeline(steps=[('Mean_Imputer', SimpleImputer()),
                ('Scaling', StandardScaler())])

# `LogisticRegressionClassifier Pipeline`

In [997]:
Logistic = make_pipeline(Processor,LogisticRegression(random_state=1))

In [998]:
Logistic

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('Mean_Imputer', SimpleImputer()),
                                 ('Scaling', StandardScaler())])),
                ('logisticregression', LogisticRegression(random_state=1))])

# `RandomForestClassifier Pipeline`

In [999]:
Forest = make_pipeline(Processor,RandomForestClassifier(random_state=1))

In [1000]:
Forest

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('Mean_Imputer', SimpleImputer()),
                                 ('Scaling', StandardScaler())])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1))])

# `VotingClassifier `

In [1001]:
voting_classifier = VotingClassifier(
    estimators=[('LogisticRegresson', Logistic), ('RandomForest', Forest)],
    voting='hard')

## `Train-Test Split`

In [1002]:
X = iris.drop('species',axis=1)
y = iris.species

In [1003]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

## `Model Training`

In [1004]:
voting_classifier.fit(X_train,y_train)

VotingClassifier(estimators=[('LogisticRegresson',
                              Pipeline(steps=[('pipeline',
                                               Pipeline(steps=[('Mean_Imputer',
                                                                SimpleImputer()),
                                                               ('Scaling',
                                                                StandardScaler())])),
                                              ('logisticregression',
                                               LogisticRegression(random_state=1))])),
                             ('RandomForest',
                              Pipeline(steps=[('pipeline',
                                               Pipeline(steps=[('Mean_Imputer',
                                                                SimpleImputer()),
                                                               ('Scaling',
                                                                StandardScaler())])),
                                              ('randomforestclassifier',
                                               RandomForestClassifier(random_state=1))]))])

## `Model Performance`

In [1005]:
y_pred = voting_classifier.predict(X_test)

In [1006]:
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9666666666666667
[[11  0  0]
 [ 0 12  1]
 [ 0  0  6]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30

